In [ ]:
import arcpy
import os
from dotenv import load_dotenv
from pathlib import Path
import json
import arcpy
from arcpy.da import InsertCursor
from shapely.geometry import Point

###################################################
########## path to geodatabase ####################
load_dotenv() # --> load in path via env variable 
raw_path:str = os.getenv('GDB_PATH')
GDB_PATH:str = r"{}".format(raw_path)
if arcpy.Exists(GDB_PATH): 
    print(f"Geodatabase exists: {GDB_PATH}")
else:
    print(f"Geodatabase DNE: {GDB_PATH}")

###################################################
#####  Spatial Refernce for Project: WGS 84  ######

SPATIAL_REF= arcpy.SpatialReference(4326) # WGS 84 (latitude/longitude)


###################################################
#######     Paths to all json files         #######
"""
All Json files contain points representing:
- grocery stores
- convenience stores
- cornerstores
- cannabis and liquor stores
- farmers markets
- fast food restaurants
- food banks
- mini marts
"""
GROCERY_JSON_PATH:object = Path(r"..\data_collection\grocery_edited.json")
CONVENIENCE_JSON_PATH:object= Path(r"..\data_collection\convenience.json")
CORNERSTORE_JSON_PATH:object = Path(r"..\data_collection\cornerstore.json")
DRUGS_NLIQUOR_JSON_PATH:object = Path(r"..\data_collection\drugs_nliquor.json")
FARM_MARKER_JSON_PATH:object = Path(r"..\data_collection\farm_market.json")
FAST_FOOD_JSON_PATH:object = Path(r"..\data_collection\fast_food.json")
FOOD_BANK_JSON_PATH:object = Path(r"..\data_collection\food_bank.json")
MINI_MART_JSON_PATH:object = Path(r"..\data_collection\mini_mart.json")

PATH_LIST:list[object] = [GROCERY_JSON_PATH,
             CONVENIENCE_JSON_PATH,
             CORNERSTORE_JSON_PATH,
             DRUGS_NLIQUOR_JSON_PATH,
             FARM_MARKER_JSON_PATH,
             FAST_FOOD_JSON_PATH,
             FOOD_BANK_JSON_PATH,
             MINI_MART_JSON_PATH
             ]

for path in PATH_LIST:
    if path.exists():
        print(f"{path.name} exists")
    else:
        print(f"{path.name} doesn't exist")

### Grocery Store Data Processing

In [2]:
"""
Grocery Store Data --> Creating a feature layer that has all the coordinates of grocery stores.

Each point will also have a 0.5 mile radius (a colored circle visually) that will signify the "reach" that a grocery store has
as a proof of concept. 
"""

### Create the grocery store feature layer ###
GROCERY_FC = "grocery_store_fc"
GROCERY_FC_PATH = os.path.join(GDB_PATH,GROCERY_FC)

arcpy.CreateFeatureclass_management(GDB_PATH,GROCERY_FC,"POINT")

### Make sure to define the coordinate system / spatial reference of any feature class that is created
arcpy.DefineProjection_management(GROCERY_FC, SPATIAL_REF)

### Verify that the grocery feature class was successfully created ###
if arcpy.Exists(GROCERY_FC_PATH):
    print("Grocery Feature Class was successfully created")
else:
    print("Grocer Feature Class DNE: Error")

Grocery Feature Class was successfully created


In [ ]:
### Check existing fields in Grocery Feature Class ###
print("Fields in the feature class:")
for field in arcpy.ListFields(GROCERY_FC_PATH):
    print(field.name)

### Add fields for storing coordinates for easier calculations later ###
arcpy.AddField_management(GROCERY_FC_PATH, "X_Coord", "DOUBLE")
arcpy.AddField_management(GROCERY_FC_PATH, "Y_Coord", "DOUBLE")
arcpy.AddField_management(GROCERY_FC_PATH,"Address","TEXT")

required_fields:list[str] = ["Address", "X_Coord", "Y_Coord"]
for required_field in required_fields:
    if any(field.name == required_field for field in arcpy.ListFields(GROCERY_FC_PATH)):
        print(f"The field '{required_field}' exists.")
    else:
        print(f"The field '{required_field}' does not exist.")


Fields in the feature class:
OBJECTID
Shape
X_Coord
Y_Coord
Address
The field 'Address' exists.
The field 'X_Coord' exists.
The field 'Y_Coord' exists.


In [8]:
### Process Grocery JSON file ###
with open(GROCERY_JSON_PATH) as file:
    GROCERY_JSON = json.load(file)

# Step 3: Insert points into the feature class from the JSON data
with arcpy.da.InsertCursor(GROCERY_FC_PATH, ["SHAPE", "Address", "X_Coord", "Y_Coord"]) as cursor:
    for address, coords in GROCERY_JSON.items():
        # Create a point geometry object using the x and y coordinates
        point = arcpy.Point(coords[1], coords[0])  # coords[1] is longitude (X), coords[0] is latitude (Y)
        
        # Prepare the row with geometry and attributes
        row = [point, address, coords[1], coords[0]]  # X_Coord is longitude, Y_Coord is latitude
        
        # Insert the row into the feature class
        cursor.insertRow(row)

# Step 4: Create a feature layer from the feature class
GROCERY_FL = "grocery_fl"
arcpy.MakeFeatureLayer_management(GROCERY_FC_PATH, GROCERY_FL)

if arcpy.Exists(GROCERY_FL):
    print("Grocery Feature Layer exists")
else:
    print("Grocery feature layer DNE:error")

Grocery Feature Layer exists
